In [2]:
import torch
import torchvision
from torch import optim
from torchvision import transforms , datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from tqdm import tqdm
import numpy as np

# Training CIFAR10 using ResNEt50 with Batch Normalization layer

In [8]:
#Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
# Load the pre-trained ResNet50 model
resnet50_bn= models.resnet50(pretrained=True)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
#put model weights on gpu
if torch.cuda.is_available():
    resnet50_bn.cuda()

In [5]:
#Load the dataset
batch_size=256
normalize = transforms.Normalize(
   mean=[0.485, 0.456, 0.406],
   std=[0.229, 0.224, 0.225]
)
trans = transforms.Compose([transforms.Resize(224), transforms.CenterCrop(224), transforms.ToTensor(),normalize])

trainset = torchvision.datasets.CIFAR10(root='.', train=True, download=True, transform=trans)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='.', train=False, download=True, transform=trans)
testloader = torch.utils.data.DataLoader(testset, batch_size=2*batch_size, shuffle=False, num_workers=2)

100%|██████████| 170498071/170498071 [00:15<00:00, 11101403.16it/s]


Extracting ./cifar-10-python.tar.gz to .
Files already downloaded and verified


In [ ]:
# Freeze the layers
for param in resnet50_bn.parameters():
    param.requires_grad = False

In [ ]:
#add classifier layer for 10 class as pretrained model is trained on imagenet which classifies 1000 classes
resnet50_bn.fc = nn.Sequential(
                      nn.Linear(2048, 256),
                      nn.ReLU(),
                      nn.Linear(256, 10)
)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet50_ln.parameters(), lr=0.001)

In [ ]:
#TRAINING THE NETWORK
from tqdm import tqdm
resnet50 = resnet50_bn.to(device)
for epoch in range(5):  # loop over the dataset multiple times
    running_loss = 0.0
    pbar = tqdm(trainloader)
    i = 0
    for data in pbar:
        # get the inputs
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = resnet50(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

        pbar.set_description("Processing epoch {:d} minibatch {:d} train loss {:.3f}".format(epoch,\
                                                            i+1, running_loss/(i+1)))
        i += 1

print('Finished Training')

Processing epoch 0 minibatch 196 train loss 0.781: 100%|██████████| 196/196 [03:03<00:00,  1.07it/s]
Processing epoch 1 minibatch 196 train loss 0.574: 100%|██████████| 196/196 [02:59<00:00,  1.09it/s]
Processing epoch 2 minibatch 196 train loss 0.535: 100%|██████████| 196/196 [02:58<00:00,  1.10it/s]
Processing epoch 3 minibatch 196 train loss 0.519: 100%|██████████| 196/196 [02:58<00:00,  1.10it/s]
Processing epoch 4 minibatch 196 train loss 0.508: 100%|██████████| 196/196 [03:05<00:00,  1.06it/s]

Finished Training


In [ ]:
torch.save(resnet50,'/content/drive/MyDrive/CV_final_project_files/resnet50_bn.pt')

# Test on Normal Images

In [ ]:
correct = 0
total = 0
i = 0
with torch.no_grad():
    pbar = tqdm(testloader)
    for data in pbar:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = resnet50(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        pbar.set_description("minibatch {:d} test accuracy {:4.2f}%".format(i+1,\
                                                            100.0*correct/total))
        i += 1

print('Accuracy of the network on the 10000 test images: %4.2f %%' % (100.0 * correct / total))

minibatch 20 test accuracy 81.87%: 100%|██████████| 20/20 [00:38<00:00,  1.91s/it]

Accuracy of the network on the 10000 test images: 81.87 %


# Code to create corrupted images

In [ ]:
from imagecorruptions import corrupt
from imagecorruptions import get_corruption_names
test_curr=np.empty((10000,32,32,3))
for i,image in enumerate(X_test):
    corruption_name = np.random.choice(curr_names)
    corrupted = corrupt(image, corruption_name=corruption_name, severity=1)
    test_curr[i]=corrupted
test_curr=test_curr.astype(int)
curr_names=get_corruption_names()

In [ ]:
curr_names

['gaussian_noise',
 'shot_noise',
 'impulse_noise',
 'defocus_blur',
 'glass_blur',
 'motion_blur',
 'zoom_blur',
 'snow',
 'frost',
 'fog',
 'brightness',
 'contrast',
 'elastic_transform',
 'pixelate',
 'jpeg_compression']

In [ ]:
curr_names.remove('glass_blur')

## Save corrupted image in Google drive

In [ ]:
output_directory = "/content/drive/MyDrive/CV_final_project_files/"

# Save the images with automatically generated file names
for i, image in enumerate(test_curr):
    # Generate a file name using a pattern or index
    file_name = os.path.join(output_directory, f"curr_image_{i:04d}.jpg")

    # Convert the NumPy array to a Pillow image
    pil_image = Image.fromarray(np.uint8(image))

    # Save the image to the specified file
    pil_image.save(file_name)

In [17]:
# Code to load the corrupt images in dataloader

In [13]:
import os
from PIL import Image
path = '/content/drive/MyDrive/CV_final_project_files/'
filenames = [filename for filename in os.listdir(path) if filename.endswith('.jpg')]
filenames.sort()

images = []
for filename in filenames[0:2000]:
    img = Image.open(os.path.join(path, filename))

    images.append(trans(img))

In [14]:
labels=[]
for tensor in testloader:
  for label in tensor[1]:
    labels.append(label.item())
labels=labels[0:2000]

In [15]:
# Create the dataset
# Create the test loader
batch_size=256
currtestset = torch.utils.data.TensorDataset(torch.stack(images), torch.tensor(labels))
currtestloader = torch.utils.data.DataLoader(currtestset, batch_size=2*batch_size, shuffle=False, num_workers=2)

# Now testing the corrupted images with our ResNET50_BN (Batch Normalization) model.

In [ ]:
from tqdm import tqdm
correct = 0
total = 0
i = 0
model.eval()
with torch.no_grad():
    pbar = tqdm(currtestloader)
    for data in pbar:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        pbar.set_description("minibatch {:d} test accuracy {:4.2f}%".format(i+1,\
                                                            100.0*correct/total))
        i += 1

print('Accuracy of the network on the 10000 test images: %4.2f %%' % (100.0 * correct / total))

minibatch 4 test accuracy 36.45%: 100%|██████████| 4/4 [00:07<00:00,  1.82s/it]

Accuracy of the network on the 10000 test images: 36.45 %


In [ ]:
torch.save(currtestset, 'testset.pt')
# Load the test dataset
# testset = torch.load('testset.pt')

# Change ResNet50 BAtch norm layers to group norm

In [18]:
# Below function changes BN layers to GN layers in ResNet50

In [3]:
def convert_bn_model_to_gn(module, num_groups=32):
    """
    Recursively traverse module and its children to replace all instances of `torch.nn.modules.batchnorm._BatchNorm` with `torch.nn.GroupNorm`.
    Args:
        module: your network module
        num_groups: num_groups of GN
    """
    mod = module
    if isinstance(module, nn.modules.batchnorm._BatchNorm):
        mod = nn.GroupNorm(num_groups, module.num_features, eps=module.eps, affine=module.affine)
        if module.affine:
            mod.weight.data = module.weight.data.clone().detach()
            mod.bias.data = module.bias.data.clone().detach()
    for name, child in module.named_children():
        mod.add_module(name, convert_bn_model_to_gn(child, num_groups=num_groups))
    del module
    return mod
# Load the pre-trained ResNet50 model


In [ ]:
resnet50_bn = models.resnet50(pretrained=True)
resnet50_gn=convert_bn_model_to_gn(module, num_groups=32)

In [4]:
# Using previously trained the model,so just loading it to test the performance
resnet50_gn=torch.load('/content/drive/MyDrive/CV_final_project_files/resnet50_gn.pt')

In [9]:
resnet50 = resnet50_gn.to(device)

In [ ]:
# We can train the model by using this code, in case not using the previously trained model
from tqdm import tqdm

for epoch in range(5):  # loop over the dataset multiple times
    running_loss = 0.0
    pbar = tqdm(trainloader)
    i = 0
    for data in pbar:
        # get the inputs
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = resnet50(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

        pbar.set_description("Processing epoch {:d} minibatch {:d} train loss {:.3f}".format(epoch,\
                                                            i+1, running_loss/(i+1)))
        i += 1

print('Finished Training')

# Testing the ResNet50_GN with Normal Images

In [11]:
correct = 0
total = 0
i = 0
resnet50.eval()
with torch.no_grad():
    pbar = tqdm(testloader)
    for data in pbar:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = resnet50(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        pbar.set_description("minibatch {:d} test accuracy {:4.2f}%".format(i+1,\
                                                            100.0*correct/total))
        i += 1

print('Accuracy of the network on the 10000 test images: %4.2f %%' % (100.0 * correct / total))

minibatch 20 test accuracy 45.19%: 100%|██████████| 20/20 [00:46<00:00,  2.31s/it]

Accuracy of the network on the 10000 test images: 45.19 %


# Testing the ResNet50_GN (Group Normalisation ) with corrupted images

In [16]:
correct = 0
total = 0
i = 0
with torch.no_grad():
    pbar = tqdm(currtestloader)
    for data in pbar:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = resnet50(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        pbar.set_description("minibatch {:d} test accuracy {:4.2f}%".format(i+1,\
                                                            100.0*correct/total))
        i += 1

print('Accuracy of the network on the 10000 test images: %4.2f %%' % (100.0 * correct / total))

minibatch 4 test accuracy 36.45%: 100%|██████████| 4/4 [00:08<00:00,  2.00s/it]

Accuracy of the network on the 10000 test images: 36.45 %


# ----------------------------------------------Finished-------------------------------------------------